# Build your Own Linear Regression

In this notebook we're going to walk through the steps involved in building a linear regression from scratch. Linear regression is a basic form of **error-based learning**. Like all error-based learning techniques, it relies on gradient descent, and hence on somewhat advanced mathematical differentiation. The intricacies of how the loss function is differentiated is beyond the scope of this course, but for those interested (and feeling brave, you can find a leaping off point at [this Stack Exchange question](https://stats.stackexchange.com/questions/278771/how-is-the-cost-function-from-logistic-regression-derivated). In this notebook I will try to explain the intution behind the approach as opposed to the internal workings.

## The Intuition

A linear regression assumes that there is some linear relationship between X and y. A very simple example will illustrate this. If X is the length of one side of a square field, and y is half of the perimeter of the field, then we can work out the value of y multiplying X by 2. The job of the linear regression is to find out what number we need to multiply X by to get the value of Y. We call this number the **weight**.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

X = np.array(np.array([[0], [1], [2], [3]]))
y = np.array(np.array([0, 2, 4, 6]))

print(len(X))
print(y)


def plot_weight(weight: float):
    plt.xlabel('X')
    plt.ylabel('y')
    plt.xlim(0, 4)
    plt.title('Relationship between X and y variables')
    plt.scatter(X, y)
    plt.plot([0, 4], [0 * weight, 4 * weight], "--")
    
plot_weight(2)

If we only dealt with perfect linear functions then linear regresions would be very easy to create (but not very useful). We could pick any two points, draw a straight line between them, and we have the correct weight mapping X to y. However, in the real world, this is not usually the case. Because we have *noise* in the dataset, and because the model will never be 100% correctly specified we'll only be able to get an approximation.

Like all error-based learning methods, Linear Regressions work on a *trial and error* basis. The approach goes like this

* Pick a random weight
* Measure how effective this weight is
* Determine whether we need to increase or decrease it to do better
* Update the weight and try again


In [ ]:
plot_weight(0.5)

We've chosen a weight of 0.5 here. When we investigate this, we can see that our line is too low for every actual value except the first. We'll therefore increase the weight and try again

In [ ]:
plot_weight(3)

We're closer now, but we've overshot it, so we'll decrease the weight and try again. Because we're closer this time we'll decrease it by less than we increased it last time

In [ ]:
plot_weight(1.8)

We're very close now, a very small increase should do the trick

In [ ]:
plot_weight(2)

## Formalising the Intution

From the above, we've seen that the general approch to error-based learning is

* make a random guess
* evaluate that guess
* improve that guess
* repeat

It's easy for us to do this using graphs, we can immediately see whether the line needs to be moved up or moved down. In order to make this easy for a computer, though, we need to formalise each of these steps. How do we evaluate a guess? How do we know whether to increase or decrease our weights?

### Evaluating a Guess: The Loss Function

A **loss function** is any function which can quantify how correct (or more precisely, incorrect) our guess is. The most common loss function for a linear regression is the **mean squared error**. You've come across the mean squared error function before, but here it is again.

$MSE = \frac{1}{N} \sum \limits_{i=1}^{N}(M(X_i) - Y_i)^2$

In order to evaluate our model, we make a prediction for each X in the training data ($X_i$), and subtract the prediction ($M(X_i)$) from the actual value, ($Y_i$). We square the results to ensure all errors are counted as positive, and divide by the total number of predictions made to get the average squared error for each prediction.

Because we've now got a number to tell us how wrong we are with any given weight, we have a solid mechanism to evaluate our guess.

It's important to remember that a loss function can be anything at all. Some more advanced algorithms will take advantage of the loss function to penalise a model for overfitting the data, so we can add terms to this function to increase it, say, if the model is using very heavy weights; or increase the penalty for certain types of errors (false positive, false negative *etc.*)


### Improving a Guess: Gradient Descent

If we were to map the possible weights we could use against the accuracy of the model, (as defined by the loss function) we'd end up with a graph looking something like this.

![Loss vs Weights](loss_curve.png)

We can see that the loss is high if our weights are much too low or much too high, but as we approach the correct weights the loss function comes down in a gently sloping curve until it reaches the minimum value. When we set our weights randomly, we'll find ourselves somewhere on the curve above. We want to make sure our next guess brings us closer to the bottom. To do this we need to know two things.

* Should we increase or decrease our weights?
* By how much should we increate or decrease our weights.

Enter differentiation. Differentiation allows us to find the slope of a curve at any given point. The slope of the curve will always point towards the minimum. The further we are away from the minimum, the steeper our curve will be. When we hit the minimum value our slope will be flat.

![Differentiating a Curve](derivative.png)

In order to find the slope of the curve at the current point we need to 

1. Differentiate the loss function
2. Plug in our current values of X

The derivative of the sum of square errors with respect to the weights is given as

![Differentiated Loss Function](derivative_loss.png)

The first part of this equation, up to the dot, is just the derivative of the loss function given above. The second part of the equation, plugs in the current values for X. It's not hugely important to understand how to do this from first principles. The important thing to note is that we're deriving our loss function and multiplying it by x to work out the derivative of the curve and find out where on that curve we're sitting.


The resulting slope will be a negative number if we need to increase our weights and a positive number if we need to decrease them. The larger the number, the steeper the slope. The steeper the slope the further we are from the correct answer, so the more we need to adjust our weights. But how much do we adjust them exactly?


### Choosing Parameters: Learning Rate and Steps

As with all error-based algorithms, we're going to keep trying over and over, incrementally improving our guess each time. Because of this, it's important that we take it slowly and don't overshoot the mark. If we increase our weights too much, we might swing right over to the other side of the curve. If it's off by a lot, then we may end up swinging from side to side in ever-increaseing arcs and never getting to the bottom. We use a parameter to slow down the rate of change of the weights. This parameter is known as the **learning rate**. The smaller the learning rate, the smaller the change in weights. This has the benefit of making sure we don't overshoot, but if it's too small it will take a very long time for our algorithm to each the bottom.

Speaking of reaching the bottom, how do we know when to stop? A linear regression takes a second parameter, the number of steps, or number of times to repeat the process. We'll keep going for a set number of steps and stop after that. This stops our algorithm from getting stuck in an infinite loop if it doesn't reach a flat point on the curve. The lower the learning rate, the more steps you need to make sure you reach the bottom.

This is simple to implement in code, and looks something like this

```python

for i in range(self.steps):
            self.W = self.W - self.learning_rate * self.calculate_gradient(Xtrain, y)
```


## Implementation Details: Matrix Multiplication and Bias


### Matrix Multiplication

In the example we opened with, we considered a very simple model which only had to deal with one feature. In more complex models, we'll have more features 

$X_1, X_2, X_3 ... X_n$

And each feature will have its own weight. 

$W_1, W_2, W_3 ... W_n$

In order to make a prediction, we'll multiply each feature in X by its corresponding weight and add all the results together to give us our predicted Y. We could do this using a loop and lots of variables, but **matrices** will do this for us much more efficiently

$[X_1, X_2, X_3] \dot [W_1, W_2, W_3]$

Will multiply each X by its corresponding W. we're doing exactly what we did above, but we're going to use numpy matrices to make sure we do it for each weight at the same time. This leads to slightly confusing notation but it's a much more efficient way to implement a linear regresssion.


### Bias or dummy variable
Finally, we need to talk about the bias term (not to be confused with the general ML term, bias). You'll notice that the examples above all go through the origin (0,0). This is because if all X values are 0, then no matter what weights we have, the result will also be 0. Many linear regressions aren't like this, and might, for example, return 10 when all Xs are 0. To handle this, a linear regression adds a dummy variable, called the bias which always evaluates to 1. The corresponding weight W_0 is the intercept in a simple linear regression. 
The he entire regression line left or right is shifted left or right from the origin by the weight W_0 associated with this bias/dummy variable.

In order to create the bias (or dummy) variable we can just create an array of 1's and stick it on as an extra feature to each item in X.  


```python
Xtrain = np.c_[np.ones(X.shape[0]), X]
```

Putting all of the above together we can now implement our linear regression model.

In [ ]:
import numpy as np

class BYOLinearRegression:
    
    # specify default values for parameters
    steps = 100000
    learning_rate = 0.001
    random_seed = 13
    
    def __init__(self, steps: int, learning_rate: float, random_seed: int):
        """Allow the user to specify number of steps and learning rate when they create a model"""
        self.steps = steps
        self.learning_rate = learning_rate
        self.random_seed = random_seed

    def calculate_gradient(self, X: np.ndarray, y: np.array):
        # This is the derivative of the sum of square errors
        # Multiplied by the current values of X to work out the slope
        # of the tangent to the error curve at our current point
        # np.dot(X, self.W) - model = prediction
        # np.dot(X, self.W) - y = error
        # np.dot(X.T, (y - np.dot(X, self.W))) = feature * error
        return 2 / X.shape[0] * np.dot(X.T, (y - np.dot(X, self.W)))


    def fit(self, X, y):
        
        # Add the bias term (this just adds a value of 1 to the beginning of every X item)
        Xtrain = np.c_[np.ones(X.shape[0]), X]
        
        # Choose random weights to start off with
        np.random.seed()
        self.W = np.random.rand((Xtrain.shape[1]))
        
        # Run for however many steps we specified
        for i in range(self.steps):
            # Subtract the slope of the curve from the current weights
            # use the learning rate to slow down the movement
            self.W = self.W + self.learning_rate * self.calculate_gradient(Xtrain, y)
            
    
    def predict(self, X):
        # multiply the X value by the current weights to get the predicted Y
        Xpred = np.c_[np.ones(X.shape[0]), X]
        return np.dot(Xpred, self.W)

## Testing the Model

The sklearn library provides a function **make_regression()** which allows us to generate synthetic data for testing. These kinds of datasets can be very useful when creating a model as we can tailor them exactly to our needs, specifying the level of noise, for example.

In [ ]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split # or we could use last week's notebook


X, Y = make_regression(n_samples = 1000, n_features = 1, n_targets = 1, bias = 2.5, noise = 40, random_state = 13)

print(X.shape, Y.shape)
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, train_size = 0.7, random_state = 44)

print(f'Training set sizes: {X_train.shape}, {Y_train.shape}')
print(f'Validation set sizes: {X_val.shape}, {Y_val.shape}')

We're going to investigate the relationship between X and Y in the training data. We can plot these using a scatterplot. The seaborn library allows us to fit a linear regression to give us an idea of what our model should be predicting

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.xlabel('X_train')
plt.ylabel('Y_train')
plt.title('Relationship between X_train and Y_train variables')
plt.scatter(X_train, Y_train)

import seaborn as sns

plt.xlabel('X_train')
plt.ylabel('Y_train')
plt.title('Relationship between X_train and Y_train variables and the best-fit line')

sns.regplot(X_train, Y_train)

We'll start off by specifying a maximum steps value of 100. Considering our learning rate is so low (only one thousandth of the slope of the curve) this is not going to be enough to fit a good model. Experiment with the parameters yourself, what happens if you set the learning rate too high?

In [ ]:
# fit the model on the train dataset

model = BYOLinearRegression(100, 0.001, 13)
model.fit(X_train, Y_train)

Y_pred = model.predict(X_val)

from sklearn.metrics import mean_squared_error
print(f'MSE on the validation set: {mean_squared_error(Y_val, Y_pred)}')

plt.xlabel('X_val')
plt.ylabel('Y')
plt.title('Real vs Predicted values comparison')

plt.scatter(X_val, Y_val)
plt.scatter(X_val, Y_pred)
